In [0]:
%pip install --upgrade databricks-sdk boto3
dbutils.library.restartPython()

In [0]:

dbutils.widgets.text("account_id", "")
dbutils.widgets.text("client_id", "")
dbutils.widgets.text("client_secret", "")

In [0]:
from databricks.sdk import AccountClient

a = AccountClient(
host="https://accounts.cloud.databricks.com/",
account_id= dbutils.widgets.get("account_id"),
client_id= dbutils.widgets.get("client_id"),
client_secret= dbutils.widgets.get("client_secret")
)

In [0]:
import time
sp = a.service_principals.create(
    active = True,
    display_name=f"sdk-{time.time_ns()}"
    )

In [0]:

sp_oauth_secret = a.service_principal_secrets.create(sp.id)
sp_oauth_secret_dict= sp_oauth_secret.as_dict()

In [0]:
from databricks.sdk.service import iam

def update_workspace_assignment(workspace_id, sp_id):
    _ = a.workspace_assignment.update(
        workspace_id=workspace_id,
        principal_id=sp_id,
        permissions=[iam.WorkspacePermission.ADMIN],
    )

In [0]:
all_workspaces = a.workspaces.list()

In [0]:
for workspace in all_workspaces:
    update_workspace_assignment(workspace.workspace_id, sp.id)

In [0]:
print(sp.id)

In [0]:
from databricks.sdk import WorkspaceClient

import json
import os

In [0]:
client = WorkspaceClient()

# Get the notebook path
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()

# Define a valid repo path outside of any existing repo
repo_path = '/security-analysis-tool'

# Create repo 
ri = client.repos.create(
    path=repo_path,
    url="https://github.com/joyaung/security-analysis-tool.git", # modified to ignore choosing of profiles
    provider="github",
)
client.repos.update(repo_id=ri.id, branch="databricks-cluster-version")